In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
cwl_dir = '../cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [11]:
template = {
    'vcf': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'inheritance_gene_list': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'pp2_gene_list': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'pathogenic_variants': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'hotspot3d_clusters': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'clinvar_alleles': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'sample': 'a_string'
}

In [12]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.charger.yaml'), 'w'))

In [15]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    'id': 'charger',
    'label': 'charger',
    'baseCommand': ['python', '/pecgs-charger/src/charger.py'],
    'inputs': [
        {
            'id': 'vcf',
            'type': 'File',
            'inputBinding': {
                'position': '1',
            }
        },
        {
            'id': 'inheritance_gene_list',
            'type': 'File',
            'inputBinding': {
                'prefix': '--inheritance-gene-list',
                'position': '0'
            }
        },
        {
            'id': 'pp2_gene_list',
            'type': 'File',
            'inputBinding': {
                'prefix': '--pp2-gene-list',
                'position': '0'
            }
        },
        {
            'id': 'pathogenic_variants',
            'type': 'File',
            'inputBinding': {
                'prefix': '--pathogenic-variants',
                'position': '0'
            }
        },
        {
            'id': 'hotspot3d_clusters',
            'type': 'File',
            'inputBinding': {
                'prefix': '--hotspot3d-clusters',
                'position': '0'
            }
        },
        {
            'id': 'clinvar_alleles',
            'type': 'File',
            'inputBinding': {
                'prefix': '--clinvar-alleles',
                'position': '0'
            }
        },
        {
            'id': 'sample',
            'type': 'string?',
            'default': 'sample',
            'inputBinding': {
                'prefix': '--sample',
                'position': '0'
            }
        },
        {
            'id': 'rare_threshold',
            'type': 'string?',
            'default': '0.0005',
            'inputBinding': {
                'prefix': '--rare-threshold',
                'position': '0'
            }
        },
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/charger/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--out-dir',
            'valueFrom': 'output'
        },
        {
            'position': 0,
            'prefix': '--format-vcf-script',
            'valueFrom': '/pecgs-charger/src/scripts/format_vcf_for_CharGer.py'
        },
        {
            'position': 0,
            'prefix': '--post-charger-script',
            'valueFrom': '/pecgs-charger/src/scripts/post_CharGer.py'
        },
        {
            'position': 0,
            'prefix': '--filter-charger-script',
            'valueFrom': '/pecgs-charger/src/scripts/filter_CharGer.py'
        },
    ],
    'outputs': [
        {
            'id': 'filtered_tsv',
            'type': 'File',
            'outputBinding': {'glob': 'output/4.filter_charger/$(inputs.sample).charged2vcf.filtered.tsv'},
        },
        {
            'id': 'rare_threshold_filtered_tsv',
            'type': 'File',
            'outputBinding': {'glob': 'output/4.filter_charger/$(inputs.sample).charged2vcf.filtered.af$(inputs.rare_threshold).tsv'},
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs-charger:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 29000,
            'coresMin': 1
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [16]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'charger.cwl'), 'w'))